# Obsersavations
1 - Many of the tweets continue to fall in as neutral and it appears that the vader sentiment analysis is conservative or this represnts media outlets being careful on tone in messages
2 - It was surprising to see that all 4 of the media mean scores were negative
3 - Plotting this type of information makes it difficult to asses any trends which is why the bar chart is a more accurate representation for this excerise



import tweepy
import json
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import seaborn as sns
import time

consumer_key = "uOF1mT0SLO9S8zMeGhqkJra8v"
consumer_secret = "mI0QoCTbCBTLo9sMj7PgdimiW7AnXQXnFE46daqt4HidleL6RD"
access_token = "969399623415562240-jPmZQZagnYGw8igneV1Dc8dzzGJSseG"
access_token_secret = "JZCG8RfGTf5HbYe8k5drmpqpQQIX2rm4YxTG4jB67lqIA"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

target_users = ["@BBCWorld", "@CBSNews", "@CNN", "@FoxNews", "@nytimes"]

compound_list = []
positive_list = []
negative_list = []
neutral_list = []
users = []
text = []
date = []
tweets_ago = []

for user in target_users:
    
    tweet_count = 0

    for x in range(5):
        public_tweets = api.user_timeline(user, page=x)

        for tweet in public_tweets:
        
            tweet_count += 1
    
            users.append(user)
            text.append(tweet['text'])
            date.append(tweet['created_at'])
            tweets_ago.append(tweet_count)
    
            compound = analyzer.polarity_scores(tweet["text"])["compound"]
            pos = analyzer.polarity_scores(tweet["text"])["pos"]
            neu = analyzer.polarity_scores(tweet["text"])["neu"]
            neg = analyzer.polarity_scores(tweet["text"])["neg"]
            
            compound_list.append(compound)
            positive_list.append(pos)
            neutral_list.append(neu)
            negative_list.append(neg)

newsd = pd.DataFrame({"Users": users, "Tweet": text, "Date": date,
    "Compound": compound_list, "Positive": positive_list,
    "Neutral": neutral_list, "Negative": negative_list,
    "Tweets Ago": tweets_ago})
newsd.head()

newsd.to_csv("newsmood.csv")

# Scatter Plot
sns.set()
plt.figure(figsize = (15,10))
plt.xlim(105,-5)
plt.ylim(-1,1)

for user in target_users:
    df = newsd.loc[newsd['Users'] == user]
    plt.scatter(df["Tweets Ago"],df["Compound"], label = user, edgecolor = "black", s=125)

plt.xlabel("Tweets Ago",fontweight='bold')
plt.ylabel("Tweet Polarity",fontweight='bold')
plt.title("Sentiment Analysis of Media Tweets (%s)" % (time.strftime("%m/%d/%Y")),fontweight='bold')

plt.legend(bbox_to_anchor = (1,1), title = 'Users')    
plt.savefig("Scatter Sentiment of Media Tweets")

plt.show()

avg_vader = newsd.groupby("Users")["Compound"].mean()
avg_vader

#bar chart
sns.set()
plt.figure(figsize = (15,10))
x_axis = np.arange(len(avg_vader))
xlabels = avg_vader.index
count = 0
for vader in avg_vader:
    plt.text(count, vader+.01, str(round(vader,2)))
    count = count + 1
plt.bar(x_axis, avg_vader, tick_label = xlabels, color = ['r', 'b', 'y', 'g', 'c'])
plt.title("Overall Sentiment of Media Tweets (%s)" % (time.strftime("%m/%d/%Y")),fontweight='bold')
plt.xlabel("Media Outlet")
plt.ylabel("Tweet Polarity")
plt.savefig("Overall Sentiment of Media Tweets")
plt.show()